# Config & Imports

In [1]:
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split

import os
import tensorflow as tf
from tensorflow import keras
import keras.layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import ReLU
from keras.callbacks import EarlyStopping

from keras.regularizers import L1L2
from keras.regularizers import L2

import utilities_LR

from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

import sys

In [2]:
config = {
   'data': {
        'n_datasets': 10_000, # the number of datasets
        
        'n_samples': 4_000, # the number of samples per dataset
        
        'n_features': 10, 
        # The total number of features. 
        # These comprise n_informative informative features, n_redundant redundant features, n_repeated duplicated features and 
        # n_features-n_informative-n_redundant-n_repeated useless features drawn at random.
        
        'n_informative': 8,
        # The number of informative features. Each class is composed of a number of gaussian clusters each located around the vertices 
        # of a hypercube in a subspace of dimension n_informative. For each cluster, informative features are drawn independently 
        # from N(0, 1) and then randomly linearly combined within each cluster in order to add covariance. The clusters are then 
        # placed on the vertices of the hypercube.
        
        'n_targets': 1,
        # The number of targets (or labels) of the classification problem.
    
        'n_clusters_per_class': 2,
        # The number of clusters per class.
        
        'class_sep': 3.0,
        # class_sepfloat, default=1.0
        # The factor multiplying the hypercube size. Larger values spread out the clusters/classes and make the classification task 
        # easier.
        
        'noise': 0,
        # flip_y (fraction of samples whose class is assigned randomly)
        
        'shuffle': True,
        # Shuffle the samples and the features.
        
        'random_state': 42,
        # Determines random number generation for dataset creation. Pass an int for reproducible output across multiple function calls.
    },
    'lambda': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': -1, # currently not used
                'val_size': 0.25,
                'random_state': None,
                'shuffle': True,
                'stratify': None
            }
        },
        'model_compile': {
            'optimizer_lambda': 'adam',
            'loss': 'mae', #tf.keras.losses.get(config['lambda_net']['loss_lambda']),
            'metrics': ['mae']
        },
        'model_fit': { # refer to keras API
            'batch_size': 32,
            'epochs': 100,
            'verbose': 0,
            'callbacks': None,
            'shuffle': True,
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'inets': {
        'data_prep': {
            'train_test_val_split': { # refer to sklearn doc
                'test_size': 0.3,
                'val_size': 0.2,
                'random_state': None,
                'shuffle': True,
                'stratify': None
            }
        },
        'model_compile': {
            
        },
        'model_fit': { # refer to keras API
            'batch_size': 256,
            'epochs': 1000,
            'verbose': 'auto',
            'callbacks': None,
            'shuffle': True,
            'class_weight': None,
            'sample_weight': None,
            'initial_epoch': 0,
            'steps_per_epoch': None,
            'validation_steps': None,
            'validation_batch_size': None,
            'validation_freq': 1
        }
    },
    'computation':{
        'n_jobs': 30,
        'use_gpu': True,
        'gpu_numbers': '5',
        'RANDOM_SEED': 1,   
    }
}

In [3]:
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'

In [4]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

os.environ['CUDA_VISIBLE_DEVICES'] = config['computation']['gpu_numbers'] if config['computation']['use_gpu'] else ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' if config['computation']['use_gpu'] else ''

os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/local/cuda-11.4' if config['computation']['use_gpu'] else ''#-10.1' #--xla_gpu_cuda_data_dir=/usr/local/cuda, 
os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=2 ,--tf_xla_enable_xla_devices' if config['computation']['use_gpu'] else ''#'--tf_xla_auto_jit=2' #, --tf_xla_enable_xla_devices

In [5]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num XLA-GPUs Available: ", len(tf.config.experimental.list_physical_devices('XLA_GPU')))

Num GPUs Available:  1
Num XLA-GPUs Available:  0


# Functions

In [6]:
def load_lambda():
    directory = utilities_LR.lambda_path_LR(config)
    
    return np.load(directory + '/lambda_weights_list.npy', allow_pickle=True)

In [7]:
def get_X_data():
    directory = utilities_LR.data_path_LR(config)
    return np.load(directory + '/X_datasets_list_dataForLambda.npy', allow_pickle=True)

In [8]:
def get_y_pred_data():
    directory = utilities_LR.lambda_path_LR(config)
    
    return np.load(directory + '/lambda_preds_list.npy', allow_pickle=True)

In [9]:
#def train_test_val_split(X, y):
#    X_train, X_test, y_train, y_test = train_test_split(X, 
#                                                        y, 
#                                                        test_size=config['inets']['data_prep']['train_test_val_split']['test_size'] + config['inets']['data_prep']['train_test_val_split']['val_size'], 
#                                                        random_state=config['inets']['data_prep']['train_test_val_split']['random_state'], 
#                                                        shuffle=config['inets']['data_prep']['train_test_val_split']['shuffle'], 
#                                                        stratify=config['inets']['data_prep']['train_test_val_split']['stratify'])
#    X_test, X_val, y__test, y_val = train_test_split(X_test, 
#                                                    y_test, 
#                                                    test_size=config['inets']['data_prep']['train_test_val_split']['val_size'] / (config['inets']['data_prep']['train_test_val_split']['test_size'] + config['inets']['data_prep']['train_test_val_split']['val_size']), 
#                                                    random_state=config['inets']['data_prep']['train_test_val_split']['random_state'], 
#                                                    shuffle=config['inets']['data_prep']['train_test_val_split']['shuffle'], 
#                                                    stratify=config['inets']['data_prep']['train_test_val_split']['stratify'])
#    return X_train, X_test, X_val, y_train, y_test, y_val
#    

# Model Training

In [10]:
X = load_lambda()

In [11]:
y_predictions = get_y_pred_data()

In [12]:
valid_feature_data = get_X_data()

In [13]:
y_predictions.shape

(10000, 4000)

In [14]:
valid_feature_data.shape

(10000, 4000, 10)

In [15]:
#valid_feature_data = np.hstack(valid_feature_data)
indexcol = np.array([np.arange(config['data']['n_datasets'], dtype=int)], ndmin=2).reshape(config['data']['n_datasets'], 1)
# print(indexcol.shape)
y_predictions = np.concatenate([indexcol, y_predictions], axis=1)

In [16]:
y_predictions.shape

(10000, 4001)

In [18]:
valid_feature_data = tf.convert_to_tensor(valid_feature_data, dtype=tf.float32)

# Custom Loss

In [19]:
@tf.function
def custom_loss(y_predictions_index, y_coef_pred):
    
    index = y_predictions_index[:, 0]
    y_true = y_predictions_index[:, 1:]
    
    index = tf.cast(index, tf.int32)
    
    X_feature_data_samples = tf.gather(valid_feature_data, index)
    
    #y_pred = tf.math.sigmoid(tf.linalg.matvec(valid_feature_data_sample, y_coef_pred))
    y_pred = tf.linalg.matvec(X_feature_data_samples, y_coef_pred)

    metric = tf.keras.losses.BinaryCrossentropy(
                                from_logits=True,
                                label_smoothing=0.0,
                                axis=-1,
                                reduction='auto',
                                name='binary_crossentropy')
    loss = metric(y_true, y_pred)
    return loss

In [20]:
#@tf.function
#def custom_loss(y_coef_dataset_index_true, y_coef_pred):
#    
#    #print("true", type(y_coef_dataset_index_true))
#    #print("true", y_coef_dataset_index_true)
#    #
#    #print("pred", type(y_coef_pred))
#    #print("pred", y_coef_pred)
#    #
#    #print(y_coef_dataset_index_true.flatten().shape)
#    #y_coef_dataset_index_true = y_coef_dataset_index_true.reshape([config['data']['n_targets']+1])
#    index = y_coef_dataset_index_true[:, 0]
#    y_coef_true = y_coef_dataset_index_true[:, 1:]
#    
#    #print("true", type(y_coef_true))
#    #print("true", y_coef_true)
#    
#    index = tf.cast(index, tf.int32)
#    #tf.print(index, output_stream=sys.stderr)
#    
#    valid_feature_data_sample = tf.gather(valid_feature_data, index)
#    
#    #y_true = tf.round(tf.matmul(valid_feature_data_sample, y_coef_true))
#    #y_pred = tf.round(tf.matmul(valid_feature_data_sample, y_coef_pred))
#    
#    y_true = tf.math.sigmoid(tf.linalg.matvec(valid_feature_data_sample, y_coef_true))
#    y_pred = tf.math.sigmoid(tf.linalg.matvec(valid_feature_data_sample, y_coef_pred))
#    
#    #tf.print(y_true, output_stream=sys.stderr)
#    #tf.print(y_pred, output_stream=sys.stderr)
#
#    #metric = tf.keras.metrics.binary_crossentropy(y_true, y_pred, from_logits=True, label_smoothing=0.0, axis=-1)
#    metric = tf.keras.losses.BinaryCrossentropy(
#                                from_logits=True,
#                                label_smoothing=0.0,
#                                axis=-1,
#                                reduction='auto',
#                                name='binary_crossentropy')
#    val = metric(y_true, y_pred)
#    #tf.print(val, output_stream=sys.stderr)
#    return val

In [21]:
# Data Prep

early_stopping = EarlyStopping(monitor='val_loss',
                                min_delta=0.001,
                                patience=25,
                                verbose=0,
                                mode='auto',
                                baseline=None,
                                restore_best_weights=True)

#X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X,
#                                                                      y)

X_train, X_val, y_train, y_val = train_test_split(X, 
                                                    y_predictions, 
                                                    test_size=config['inets']['data_prep']['train_test_val_split']['val_size'], 
                                                    random_state=config['inets']['data_prep']['train_test_val_split']['random_state'], 
                                                    shuffle=config['inets']['data_prep']['train_test_val_split']['shuffle'], 
                                                    stratify=config['inets']['data_prep']['train_test_val_split']['stratify'])


# Model Def
model = Sequential()
#model.add(BatchNormalization())
model.add(Dense(4096, activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
#model.add(Dense(2500, activation='relu'))
#model.add(Dense(1500, activation='linear'))
#model.add(Dense(500, activation='relu'))
model.add(Dense(config['data']['n_features'], activation='linear'))


opt = tf.keras.optimizers.Adam(
    learning_rate=0.0001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name='Adam',)

#opt = tf.keras.optimizers.SGD(
#    learning_rate=0.01,
#    momentum=0.0,
#    nesterov=False,
#    name='SGD'
#)

model.compile(optimizer=opt, loss=custom_loss, metrics=[])
model.build(input_shape=X_train.shape)
model.summary()

# Model fit
history = model.fit(x=X_train,
                    y=y_train,
                    batch_size=config['inets']['model_fit']['batch_size'],
                    epochs=config['inets']['model_fit']['epochs'],
                    verbose=config['inets']['model_fit']['verbose'],
                    #callbacks=[early_stopping],
                    validation_data=(X_val, y_val),
                    shuffle=config['inets']['model_fit']['shuffle'],
                    class_weight=config['inets']['model_fit']['class_weight'],
                    sample_weight=config['inets']['model_fit']['sample_weight'],
                    initial_epoch=config['inets']['model_fit']['initial_epoch'],
                    steps_per_epoch=config['inets']['model_fit']['steps_per_epoch'],
                    validation_steps=config['inets']['model_fit']['validation_steps'],
                    validation_batch_size=config['inets']['model_fit']['validation_batch_size'],
                    validation_freq=config['inets']['model_fit']['validation_freq'],
                    workers=config['computation']['n_jobs'],
                    #use_multiprocessing=True
                   )
print(history.history['val_loss'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (8000, 4096)              29745152  
                                                                 
 dense_1 (Dense)             (8000, 2048)              8390656   
                                                                 
 dense_2 (Dense)             (8000, 1024)              2098176   
                                                                 
 dense_3 (Dense)             (8000, 512)               524800    
                                                                 
 dense_4 (Dense)             (8000, 10)                5130      
                                                                 
Total params: 40,763,914
Trainable params: 40,763,914
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
32/32 [========================

In [22]:
def save_model(model):
    path = utilities_LR.inet_path_LR(config)
    
    Path(path).mkdir(parents=True, exist_ok=True)
    model.save(path + '/modelKeras')

In [23]:
save_model(model)

INFO:tensorflow:Assets written to: data_LR/nda10000_nsa4000_nfe10_nin8_nta1_ncc2_sep3.0_noi0_shuTrue_ran42/tsi-1_vsi0.25_ranNone_shuTrue_strNone_bat32_epo100_shuTrue_claNone_samNone_ini0_steNone_vstNone_vbsNone_vfr1/tsi0.3_vsi0.2_ranNone_shuTrue_strNone_bat256_epo1000_shuTrue_claNone_samNone_ini0_steNone_vstNone_vbsNone_vfr1/modelKeras/assets
